In [88]:
import pandas as pd

# Charger le fichier nettoyé dans un DataFrame
df_clean = pd.read_csv('house_data_cleaned.csv')

In [85]:
y = df_clean["prix_vente"]
X = df_clean.drop(["date_vente", "id","nb_etage" ,"vue_sur_mer", "qualite_vue", "superficie_sur_sol", "superficie_sous_sol", "annee_construction", "annee_renovation", "code_postal","surface_totale15"], axis=1)
# Séparer les données en deux parties: entraînement et test
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

knn.score(X_test, y_test)

0.537589636826278

In [86]:
# # Créer une pipeline de données pour automatiser le processus de préparation des données
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# # Spécifier les colonnes pour la transformation de feature
# num_features = ['surface_habitable', 'surface_totale', 'nb_sdb', 'surface_habitable15', 'qualite_globale', 'longitude', 'latitude']

# # Créer une instance de la classe ColumnTransformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), num_features), # Normalisation
#         ('poly', PolynomialFeatures(degree=2), num_features) # Création de variables polynomiales
#     ])

# # Créer la pipeline de données en chaînant les étapes de prétraitement
# pipeline = Pipeline([
#     ('preprocessor', preprocessor)
# ])

# # Appliquer la pipeline de préparation de données sur le jeu de données d'entraînement
# train_prepared = pipeline.fit_transform(train_data)

# # Créer un DataFrame à partir du jeu de données préparé
# df_train_prepared = pd.DataFrame(train_prepared, columns=pipeline['preprocessor'].get_feature_names())

# # Sauvegarder le jeu de données préparé sous forme de fichier CSV
# df_train_prepared.to_csv('house_data_prepared.csv')



In [90]:
from sklearn.model_selection import train_test_split
y = df_clean['prix_vente']
X = df_clean[["surface_habitable","nb_sdb","qualite_globale","latitude","longitude","surface_totale","vue_sur_mer"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state=42)

# pre-processing de X_train
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
X_train_min_max = minmax.fit_transform(X_train)

# Entrainement sur X_train
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=4)
trained_model = model.fit(X_train_min_max,y_train)

# pre-processing de X_test
X_test_min_max = minmax.transform(X_test)

# scoring
trained_model.score(X_test_min_max,y_test)


0.8364543934568593

In [94]:
from sklearn.model_selection import train_test_split
y = df_clean['prix_vente']
X = df_clean[["surface_habitable","nb_sdb","qualite_globale","latitude","longitude","surface_totale","vue_sur_mer"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)


# Création du pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor

minmax = MinMaxScaler()
knn_4 = KNeighborsRegressor(n_neighbors=4)

pipe = Pipeline([
     ('minmax', minmax),
     ('knn', knn_4)
])
# Entrainement sur X_train
trained_pipe = pipe.fit(X_train,y_train)

# prediction sur X_test
trained_pipe.predict(X_test)

# scoring sur X_test
trained_pipe.score(X_test,y_test)

0.8364543934568593

In [95]:
trained_pipe["minmax"].transform(X_test)

array([[0.15136054, 0.28125   , 0.58333333, ..., 0.29651163, 0.00719161,
        0.        ],
       [0.22193878, 0.375     , 0.58333333, ..., 0.19435216, 0.0053338 ,
        0.        ],
       [0.29591837, 0.3125    , 0.83333333, ..., 0.32392027, 0.00890941,
        0.        ],
       ...,
       [0.18231293, 0.3125    , 0.58333333, ..., 0.19601329, 0.00333255,
        0.        ],
       [0.16666667, 0.21875   , 0.58333333, ..., 0.33305648, 0.00823517,
        0.        ],
       [0.19387755, 0.3125    , 0.58333333, ..., 0.18521595, 0.00940758,
        0.        ]])

In [99]:
from sklearn.model_selection import train_test_split
y = df_clean['prix_vente']
X = df_clean[["surface_habitable","nb_sdb","qualite_globale","latitude","longitude","surface_totale","vue_sur_mer"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)


# On cree un pipeline de proprocessing pour les variables numériques
numeric_features = ["surface_habitable","nb_sdb","surface_totale","latitude","longitude"]

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('stdscaler', MinMaxScaler()),  # moyenne nulle et écart type = 1 -> Reg, SVM, PCA
        ])

# On cree un pre-processeur pour les variables catégorielles
categorial_features = [ "qualite_globale","vue_sur_mer"]

from sklearn.preprocessing import OneHotEncoder
categorical_transformer = OneHotEncoder(sparse=True)


# a l'aide de la classe ColumnTransformer, 
# on déclare à quelles variables on applique quel transformer
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorial_features)
    ]
)
#On obtient un pipeline de preprocessing qu'on peut utiliser dans un pipeline d'entainement
from sklearn.neighbors import KNeighborsRegressor
knn_4 = KNeighborsRegressor(n_neighbors=4)

pipe = Pipeline([
     ('prep', preprocessor),
     ('knn', knn_4)
])
# Entrainement sur X_train
trained_pipe = pipe.fit(X_train,y_train)

# prediction sur X_test
trained_pipe.predict(X_test)

# scoring sur X_test
trained_pipe.score(X_test,y_test)

/home/apprenant/miniconda3/envs/machine_learning/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.8483756101087857